In [1]:
from pathlib import Path

from flexx import flx
import pandas as pd
from pscript.stubs import d3

[I 15:28:18 flexx.app] Asset store collected 2 new modules.


In [2]:
try:
    flx.assets.associate_asset(__name__, 'https://d3js.org/d3.v5.min.js')
except ValueError:
    pass

In [3]:
DATA_ROOT = Path('..') / 'data'

In [4]:
dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])

sensor_df.to_csv('sensor_df.csv')

### Make data for plotting

In [5]:
# We join the participant name with the activity label of the row
# to get ['d1p01M-bed', 'd1p01M-bed', ...] etc.
values = sensor_df['participant'].str.cat(sensor_df['activity_label'], sep='-')

# Using shift to compare consecutive values we can obtain a boolean series
# which is `True` at the start of every activity, and `False` otherwise.
activity_start = values.shift(1) != values

# The cumsum trick converts the boolean values to integers,
# so `True` becomes `1` and `False` becomes `0`. 
# As `activity_start` is `True` at the start of every activity and `False`
# otherwise, successive activities in the data get an different (increasing)
# number:
#
#    list(pd.Series([True, False, True, True]).cumsum()) # <- [1, 1, 2, 3]
#
sensor_df['activity_block'] = activity_start.cumsum()

# Create a data structure for plotting.
# Must make sure all data types are python native.
plot_data = {
    'time_range': [sensor_df['time'].min(), sensor_df['time'].max()],
    'participants': {}
}
for participant, participant_df in sensor_df.groupby(['participant']):

    plot_data['participants'][participant] = {
        'activities': [],
        'time': [participant_df['time'].min(), participant_df['time'].max()]
    }
    for _, activity_df in participant_df.groupby(['activity_block']):
        activity = {
            'start': activity_df['time'].min(),
            'end': activity_df['time'].max(),
            'duration': activity_df['time'].max() - activity_df['time'].min(),
            'activity': int(activity_df['activity'].unique()[0]),
            'activity_label': activity_df['activity_label'].unique()[0],
            'sensor_data': {
                'time': list(activity_df['time']),
                'front': list(activity_df['front']),
                'lateral': list(activity_df['lateral']),
                'vertical': list(activity_df['vertical'])
            }
        }
        plot_data['participants'][participant]['activities'].append(activity)
            


### Create a widget for the plot

In [143]:
SENSOR_COLORS = {
  'front': '#1f77b4',
  'vertical': '#ff7f0e',
  'lateral': '#2ca02c',
}

ACTIVITY_COLORS = {
  '4': '#fed9a6',
  '1': '#b3cde3',
  '2': '#ccebc5',
  '3': '#decbe4',
}

BUTTON_COLORS = {
    'fg': '#ffffff',
    'bg': '#444444'
}


participant_ids = list(plot_data['participants'].keys())

class ActivityPlot(flx.Widget):
    
    participants = flx.AnyProp(participant_ids, settable=False)
    nparticipants = flx.IntProp(len(participant_ids), settable=False)
    
    participant_index = flx.IntProp(0, settable=True)
    share_x_axis = flx.BoolProp(False, settable=True)
    smoothing = flx.IntProp(0, settable=True)
    
    def init(self):
        self.node.id = self.id
        window.setTimeout(self.plot, 500)
        window.onorientationchange = lambda: window.location.reload()

    @flx.emitter
    def key_down(self, e):
        """Overload key_down emitter to prevent browser scroll."""
        ev = self._create_key_event(e)
        if ev.key.startswith('Arrow'):
            e.preventDefault()
        return ev
    
    @flx.reaction('key_down')
    def _handle_change_participant(self, *events):
        for ev in events:
            if ev.modifiers:
                continue
            elif ev.key == 'ArrowRight':
                self.next_participant()
            elif ev.key == 'ArrowLeft':
                self.prev_participant()
    
    @flx.reaction('participant_index')
    def on_participant_index_change(self):
        """Executed when the `participant_index` property changes."""
        self.redraw_plot()
    
    @flx.reaction('share_x_axis')
    def on_share_x_axis_change(self):
        """Executed when the `share_x_axis` property changes."""
        self.redraw_plot()
    
    @flx.reaction('smoothing')
    def on_smoothing_change(self):
        """Executed when the smoothing constant changes."""
        self.redraw_plot()
        
    def next_participant(self):
        """Change to the next participant."""
        index = self.participant_index
        index = (index + 1) % self.nparticipants
        self.set_participant_index(index)      
    
    def prev_participant(self):
        """Change to the previous participant."""
        index = self.participant_index
        index = index - 1
        if index < 0:
            index = self.nparticipants - 1
        self.set_participant_index(index)
        
    def change_share_x_axis(self, value):
        """Change the sharing of the x-axis."""
        self.set_share_x_axis(value)
    
    def change_smoothing(self, value):
        """Change the smoothing of the data."""
        self.set_smoothing(value)
        
    def clear_plot(self):
        """Clear (remove) the plot."""
        root = d3.select('#' + self.id)
        root.select('svg').remove()
    
    def redraw_plot(self):
        """Redraw (i.e., clear and draw) the plot."""
        self.clear_plot()
        self.plot()
    
    def plot(self):
        """Plot a participants data."""
        w, h = self.size
    
        margins = {'top': 0, 'bottom': 50, 'left': 40, 'right': 40}
        vmargin = 20
        hmargin = 40
        axis_height = h - margins['top'] - margins['bottom']
        axis_width = w - margins['left'] - margins['right']
        self.clear_plot()
        
        root = d3.select('#' + self.id)
        
        svg = root.append('svg').attr('width', w).attr('height', h)
        axis = svg.append('g').attr('transform', 'translate({},{})'.format(margins['left'], margins['top']))
        (axis.append('rect')
             .attr('width', axis_width)
             .attr('height', axis_height)
             .style('fill', 'transparent'))
        time_range = plot_data['time_range']
        participant = self.participants[self.participant_index]
        data = plot_data['participants'][participant]
        
        max_time = time_range[1] if self.share_x_axis else max(data['time'])
        
        x = d3.scaleLinear().domain([0, max_time]).range([0, axis_width])
        xAxis = (axis.append('g')
                 .attr('transform', f'translate(0,{axis_height})')
                 .call(d3.axisBottom(x)))
        
        y = d3.scaleLinear().domain([-2, 2]).range([axis_height, 0])
        
        paths = []
        markers = []
        rectangles = []
        labels = []
        
        nactivities = len(data['activities'])
        for i, activity_data in enumerate(data['activities']):
            
            # Plot activity rectangles
            rectangle = (axis.datum(activity_data)
                    .append('rect')
                    .attr('x', lambda d: x(d['start']))
                    .attr('y', 0)
                    .attr('width', lambda d: max(x(d['end']) - x(d['start']), 1))
                    .attr('height', axis_height)
                    .attr('fill', lambda d: ACTIVITY_COLORS[d['activity']])
                    .attr('opacity', 0.333))
            rectangles.append(rectangle)
            
    
            # Plot sensor data
            for sensor in ['front', 'lateral', 'vertical']:
                
                smoothing = self.smoothing + 1
                original = activity_data['sensor_data'][sensor]
                if smoothing > 1:
                    # Moving average
                    smoothed = [sum(original[i:i+smoothing])/smoothing for i in range(len(original))]
                else:
                    smoothed = original
                    
                if smoothing > 1:
                    points = list(zip(activity_data['sensor_data']['time'], original))
                    path = (axis.append('path')
                        .datum(points)
                        .attr('fill', 'none')
                        .attr('stroke', SENSOR_COLORS[sensor])
                        .attr('stroke-width', 1.5)
                        .attr('opacity', '0.25')
                        .attr('d', d3.line().x(lambda d: x(d[0])).y(lambda d: y(d[1])) ))
                    paths.append(path)
                
                points = list(zip(activity_data['sensor_data']['time'], smoothed))
                path = (axis.append('path')
                    .datum(points)
                    .attr('fill', 'none')
                    .attr('stroke', SENSOR_COLORS[sensor])
                    .attr('stroke-width', 1.5)
                    .attr('d', d3.line().x(lambda d: x(d[0])).y(lambda d: y(d[1])) ))
                paths.append(path)
                
                marks = (axis.selectAll('dot')
                    .data(points)
                    .enter()
                    .append('circle')
                    .attr('cx', lambda d: x(d[0]))
                    .attr('cy', lambda d: y(d[1]))
                    .attr('r', 2)
                    .style('opacity', 0.5)
                    .style('fill', SENSOR_COLORS[sensor]))
                markers.append(marks)
                
            # Plot activity_labels
            label = (axis.datum(activity_data)
                   .append('text')
                   .attr('x', lambda d: (x(d['end']) + x(d['start'])) / 2)
                   .attr('y',  axis_height - (i + 0.5) / nactivities * axis_height)
                   .attr('text-anchor', 'middle')
                   .style('font-size', 10)
                   .text(lambda d: d['activity_label']))
            labels.append(label)
                
        # Add participant label
        (axis.append('text')
           .attr('x', w / 2 )
           .attr('y',  axis_height - 10)
           .attr('text-anchor', 'middle')
           .style('font-size', 10)
           .text('{} {}/{}'.format(
               participant,
               self.participant_index + 1,
               self.nparticipants)))
            
        
        # Function executed when updating plot for zoom/scroll
        def update_chart():
            
            newX = d3.event.transform.rescaleX(x)
            newY = y
            xAxis.call(d3.axisBottom(newX))
            
            for path in paths:
                path.attr('d', d3.line().x(lambda d: newX(d[0])).y(lambda d:y(d[1])))
                
            for marks in markers:
                marks.attr('cx', lambda d: newX(d[0])).attr('cy', lambda d:y(d[1]))
            
            for rectangle in rectangles:
                (rectangle.attr('x', lambda d: newX(d['start']))
                 .attr('width', lambda d: newX(d['end']) - newX(d['start'])))
            
            for label in labels:
                (label.attr('x',  lambda d: (newX(d['end']) + newX(d['start'])) / 2))

        zoom = (d3.zoom()
            .scaleExtent([0.05, 20]) 
            .extent([[0, 0], [w, h]])
            .on('zoom', update_chart))
        
        (axis.append('rect')
            .attr('width', w - 2 * hmargin)
            .attr('height', h - 2 * vmargin)
            .style('fill', 'none')
            .style('pointer-events', 'all')
            .attr('transform', 'translate(0,0)')
            .call(zoom))
        
        # Add some unnecessarily fancy svg buttons.
        prevButton = (axis.append('g')
            .attr('transform', f'translate({0},{axis_height - 35}) scale(-1)'))
        
        (prevButton.append('circle')
            .attr('cx', 1.5)
            .attr('cy', 0)
            .attr('r', 30)
            .style('opacity', '0.125')
            .style('fill', BUTTON_COLORS['bg'])
            .on('click', self.prev_participant))
        (prevButton.append('path')
                .attr('d', 'M0,0 L-10,10 0,20 20,0, 0,-20, -10,-10 Z')
                .style('opacity', '0.333')
                .style('fill', BUTTON_COLORS['fg'])
                .style('pointer-events', 'none'))
        
        nextButton = (axis.append('g')
            .attr('transform', f'translate({axis_width},{axis_height - 35})'))
        
        (nextButton.append('circle')
            .attr('cx', 1.5)
            .attr('cy', 0)
            .attr('r', 30)
            .style('opacity', '0.125')
            .style('fill', BUTTON_COLORS['bg'])
            .on('click', self.next_participant))
        (nextButton.append('path')
                .attr('d', 'M0,0 L-10,10 0,20 20,0, 0,-20, -10,-10 Z')
                .style('opacity', '0.333')
                .style('fill', BUTTON_COLORS['fg'])
                .style('pointer-events', 'none'))

        
class PlotLayout(flx.Widget):
    """UI Container Widget"""
    def init(self):
        with flx.GridLayout(ncolumns=1, style='margin: 0; padding: 0;'):
            
            with flx.GridLayout(ncolumns=2, flex=(1, 0)):
                with flx.HBox(flex=(1,0), style='align-items: center'):
                    self.share_x_axis_checkbox = flx.CheckBox(text='Share x axis', flex=(1,1))
                with flx.HBox(flex=(1,0), style='align-items: center'):
                    flx.Label(text='Smoothing')
                    self.smooth_slider = flx.Slider(min=0, max=20, step=1, flex=(1, 0))
            self.plot = ActivityPlot(flex=(1, 1))
            
    @flx.reaction('share_x_axis_checkbox.checked')
    def change_share_x_axis(self, *events):
        for ev in events:
            self.plot.change_share_x_axis(ev['new_value'])
            
    @flx.reaction('smooth_slider.user_done')
    def change_smoothing(self, *events):
        for ev in events:
            self.plot.change_smoothing(ev['new_value'])

### Display plot widget

In [144]:
# Save the app as a self-contained HTML page
# and then load in to jupyter using an iframe:
app = flx.App(PlotLayout)
html = f'{app.cls.__name__}.html'
app.export(html, link=0)
from IPython.display import IFrame

IFrame(html, width=800, height=600)

[I 17:56:33 flexx.app] Exported standalone app to '/Users/mwibrow/github/motiondetection/notebooks/PlotLayout.html'
